Here we’ll focus on creating the corpus, and in the next section we’ll tackle the semantic search application. To keep things meta, we’ll use the GitHub issues associated with a popular open source project: 🤗 Datasets! Let’s take a look at how to get the data and explore the information contained in these issues.

### Getting the data

You can find all the issues in 🤗 Datasets by navigating to the repository’s [Issues tab](https://github.com/huggingface/datasets/issues). As shown in the following screenshot, at the time of writing there were 331 open issues and 668 closed ones.

To download all the repository’s issues, we’ll use the [GitHub REST API](https://docs.github.com/en/rest) to poll the [Issues endpoint](https://docs.github.com/en/rest/reference/issues#list-repository-issues). This endpoint returns a list of JSON objects, with each object containing a large number of fields that include the title and description as well as metadata about the status of the issue and so on.

A convenient way to download the issues is via the requests library, which is the standard way for making HTTP requests in Python. You can install the library by running:

In [2]:
import requests

url = "https://api.github.com/repos/huggingface/datasets/issues?page=1&per_page=1"

response = requests.get(url)

response.status_code

200

In [3]:
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/7179',
  'repository_url': 'https://api.github.com/repos/huggingface/datasets',
  'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/7179/labels{/name}',
  'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/7179/comments',
  'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/7179/events',
  'html_url': 'https://github.com/huggingface/datasets/pull/7179',
  'id': 2552387980,
  'node_id': 'PR_kwDODunzps585Jcd',
  'number': 7179,
  'title': 'Support Python 3.11',
  'user': {'login': 'albertvillanova',
   'id': 8515462,
   'node_id': 'MDQ6VXNlcjg1MTU0NjI=',
   'avatar_url': 'https://avatars.githubusercontent.com/u/8515462?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/albertvillanova',
   'html_url': 'https://github.com/albertvillanova',
   'followers_url': 'https://api.github.com/users/albertvillanova/followers',
   'following_url': 'http

As described in the GitHub [documentation](https://docs.github.com/en/rest/overview/resources-in-the-rest-api#rate-limiting), unauthenticated requests are limited to 60 requests per hour. Although you can increase the `per_page` query parameter to reduce the number of requests you make, you will still hit the rate limit on any repository that has more than a few thousand issues. So instead, you should follow GitHub’s [instructions](https://docs.github.com/en/github/authenticating-to-github/creating-a-personal-access-token) on creating a personal access token so that you can boost the rate limit to 5,000 requests per hour. Once you have your token, you can include it as part of the request header:

In [5]:
with open('/home/loc/Documents/keys/GH_key.txt') as f:
    GITHUB_TOKEN = f.read().strip()

In [6]:
headers = {"Authorization": f"token {GITHUB_TOKEN}"}

Now that we have our access token, let’s create a function that can download all the issues from a GitHub repository:

In [7]:
import time
import math
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm


def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=10_000,
    rate_limit=5_000,
    issues_path=Path("."),
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    batch = []
    all_issues = []
    per_page = 100  # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        # Query with state=all to get both open and closed issues
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # Flush batch for next time period
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

Now when we call `fetch_issues()` it will download all the issues in batches to avoid exceeding GitHub’s limit on the number of requests per hour; the result will be stored in a `repository_name-issues.jsonl` file, where each line is a JSON object the represents an issue. Let’s use this function to grab all the issues from 🤗 Datasets:

In [8]:
# Depending on your internet connection, this can take several minutes to run...
fetch_issues()

  0%|          | 0/100 [00:00<?, ?it/s]

Reached GitHub rate limit. Sleeping for one hour ...
Downloaded all the issues for datasets! Dataset stored at ./datasets-issues.jsonl


In [17]:
import json
from datasets import load_dataset

In [24]:
!pip install -q jsonl2json

In [27]:
from jsonl2json import JsonlToJsonFormatter

jsonl = JsonlToJsonFormatter('datasets-issues.jsonl', 'datasets-issues.json')
jsonl.to_json()

In [28]:
issues_dataset = load_dataset("json", data_files="datasets-issues.json", split="train")
issues_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'draft', 'pull_request', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason'],
    num_rows: 7131
})

### Cleaning up the data

The above snippet from GitHub’s documentation tells us that the `pull_request` column can be used to differentiate between issues and pull requests. Let’s look at a random sample to see what the difference is. As we did in section 3, we’ll chain `Dataset.shuffle()` and `Dataset.select()` to create a random sample and then zip the `html_url` and pull_request columns so we can compare the various URLs:

In [29]:
sample = issues_dataset.shuffle(seed=666).select(range(3))

# Print out the URL and pull request entries
for url, pr in zip(sample["html_url"], sample["pull_request"]):
    print(f">> URL: {url}")
    print(f">> Pull request: {pr}\n")

>> URL: https://github.com/huggingface/datasets/pull/6998
>> Pull request: {'diff_url': 'https://github.com/huggingface/datasets/pull/6998.diff', 'html_url': 'https://github.com/huggingface/datasets/pull/6998', 'merged_at': '2024-06-25T08:13:42Z', 'patch_url': 'https://github.com/huggingface/datasets/pull/6998.patch', 'url': 'https://api.github.com/repos/huggingface/datasets/pulls/6998'}

>> URL: https://github.com/huggingface/datasets/issues/3615
>> Pull request: None

>> URL: https://github.com/huggingface/datasets/pull/6429
>> Pull request: {'diff_url': 'https://github.com/huggingface/datasets/pull/6429.diff', 'html_url': 'https://github.com/huggingface/datasets/pull/6429', 'merged_at': '2023-11-28T16:03:43Z', 'patch_url': 'https://github.com/huggingface/datasets/pull/6429.patch', 'url': 'https://api.github.com/repos/huggingface/datasets/pulls/6429'}



Here we can see that each pull request is associated with various URLs, while ordinary issues have a `None` entry. We can use this distinction to create a new `is_pull_request` column that checks whether the `pull_request` field is `None` or not:

In [30]:
issues_dataset = issues_dataset.map(
    lambda x: {'is_pull_request': False if x['pull_request'] is None else True}
)

Map:   0%|          | 0/7131 [00:00<?, ? examples/s]

Although we could proceed to further clean up the dataset by dropping or renaming some columns, it is generally a good practice to keep the dataset as “raw” as possible at this stage so that it can be easily used in multiple applications.

Before we push our dataset to the Hugging Face Hub, let’s deal with one thing that’s missing from it: the comments associated with each issue and pull request. We’ll add them next with — you guessed it — the GitHub REST API!

### Augmenting the dataset

As shown in the following screenshot, the comments associated with an issue or pull request provide a rich source of information, especially if we’re interested in building a search engine to answer user queries about the library.

![img](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter5/datasets-issues-comment.png)

The GitHub REST API provides a [Comments endpoint](https://docs.github.com/en/rest/reference/issues#list-issue-comments) that returns all the comments associated with an issue number. Let’s test the endpoint to see what it returns:



In [31]:
issue_number = 2792
url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
response = requests.get(url, headers=headers)
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/comments/897594128',
  'html_url': 'https://github.com/huggingface/datasets/pull/2792#issuecomment-897594128',
  'issue_url': 'https://api.github.com/repos/huggingface/datasets/issues/2792',
  'id': 897594128,
  'node_id': 'IC_kwDODunzps41gDMQ',
  'user': {'login': 'bhavitvyamalik',
   'id': 19718818,
   'node_id': 'MDQ6VXNlcjE5NzE4ODE4',
   'avatar_url': 'https://avatars.githubusercontent.com/u/19718818?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/bhavitvyamalik',
   'html_url': 'https://github.com/bhavitvyamalik',
   'followers_url': 'https://api.github.com/users/bhavitvyamalik/followers',
   'following_url': 'https://api.github.com/users/bhavitvyamalik/following{/other_user}',
   'gists_url': 'https://api.github.com/users/bhavitvyamalik/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/bhavitvyamalik/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/

We can see that the comment is stored in the `body` field, so let’s write a simple function that returns all the comments associated with an issue by picking out the `body` contents for each element in `response.json()`:

In [33]:
def get_comments(issue_number):
    url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
    response = requests.get(url, headers=headers)
    return [r["body"] for r in response.json()]


# Test our function works as expected
get_comments(2792)

["@albertvillanova my tests are failing here:\r\n```\r\ndataset_name = 'gooaq'\r\n\r\n    def test_load_dataset(self, dataset_name):\r\n        configs = self.dataset_tester.load_all_configs(dataset_name, is_local=True)[:1]\r\n>       self.dataset_tester.check_load_dataset(dataset_name, configs, is_local=True, use_local_dummy_data=True)\r\n\r\ntests/test_dataset_common.py:234: \r\n_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ \r\ntests/test_dataset_common.py:187: in check_load_dataset\r\n    self.parent.assertTrue(len(dataset[split]) > 0)\r\nE   AssertionError: False is not true\r\n```\r\nWhen I try loading dataset on local machine it works fine. Any suggestions on how can I avoid this error?",
 'Thanks for the help, @albertvillanova! All tests are passing now.']

This looks good, so let’s use `Dataset.map()` to add a new `comments` column to each issue in our dataset:

```python
# Depending on your internet connection, this can take a few minutes...
issues_with_comments_dataset = issues_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)
```

In [41]:
# Let get a small dataset with 100 samples first
small_issues_with_comments_dataset = issues_dataset.select(range(100))
small_issues_with_comments_dataset = small_issues_with_comments_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Now that we have our augmented dataset, it’s time to push it to the Hub so we can share it with the community! Uploading a dataset is very simple: just like models and tokenizers from 🤗 Transformers, we can use a `push_to_hub()` method to push a dataset. To do that we need an authentication token, which can be obtained by first logging into the Hugging Face Hub with the `notebook_login()` function:

In [42]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# you can log in via the CLI instead
! huggingface-cli login

In [43]:
dataset_name = "github-issues-100"
small_issues_with_comments_dataset.push_to_hub(dataset_name)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/locchuong/github-issues-100/commit/1e17d017f6e2a1ab181fc41def691957bbea0888', commit_message='Upload dataset', commit_description='', oid='1e17d017f6e2a1ab181fc41def691957bbea0888', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/locchuong/github-issues-100', endpoint='https://huggingface.co', repo_type='dataset', repo_id='locchuong/github-issues-100'), pr_revision=None, pr_num=None)

In [47]:
remote_dataset = load_dataset("locchuong/github-issues-100", split="train")
remote_dataset

README.md:   0%|          | 0.00/6.86k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/263k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'draft', 'pull_request', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'is_pull_request'],
    num_rows: 100
})

###  Creating a dataset card

Well-documented datasets are more likely to be useful to others (including your future self!), as they provide the context to enable users to decide whether the dataset is relevant to their task and to evaluate any potential biases in or risks associated with using the dataset.

On the Hugging Face Hub, this information is stored in each dataset repository’s README.md file. There are two main steps you should take before creating this file:

1. Use the [datasets-tagging application](https://huggingface.co/datasets/tagging/) to create metadata tags in YAML format. These tags are used for a variety of search features on the Hugging Face Hub and ensure your dataset can be easily found by members of the community. Since we have created a custom dataset here, you’ll need to clone the `datasets-tagging` repository and run the application locally. Here’s what the interface looks like:

![img1](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter5/datasets-tagger.png)

2. Read the 🤗 Datasets guide on creating informative dataset cards and use it as a template.

You can create the README.md file directly on the Hub, and you can find a template dataset card in the `lewtun/github-issues` dataset repository. A screenshot of the filled-out dataset card is shown below.

![img2](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter5/dataset-card.png)


That’s it! We’ve seen in this section that creating a good dataset can be quite involved, but fortunately uploading it and sharing it with the community is not. In the next section we’ll use our new dataset to create a semantic search engine with 🤗 Datasets that can match questions to the most relevant issues and comments.

✏️ Try it out! Go through the steps we took in this section to create a dataset of GitHub issues for your favorite open source library (pick something other than 🤗 Datasets, of course!). For bonus points, fine-tune a multilabel classifier to predict the tags present in the labels field.

**Build 1k sample dataset**

In [48]:
# Let create a dataset with 1000 samples
med_issues_with_comments_dataset = issues_dataset.select(range(1000))
med_issues_with_comments_dataset = med_issues_with_comments_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)

# Updload to 
dataset_name = "github-issues-1k"
med_issues_with_comments_dataset.push_to_hub(dataset_name)

print("Upload Successful!")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/locchuong/github-issues-1k/commit/4b4791f0344c38bbc7290e25fef97b31b11d3b53', commit_message='Upload dataset', commit_description='', oid='4b4791f0344c38bbc7290e25fef97b31b11d3b53', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/locchuong/github-issues-1k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='locchuong/github-issues-1k'), pr_revision=None, pr_num=None)